In [1]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.misc import face
from scipy.ndimage import affine_transform
import sys
import cv2

In [ ]:
# We are doing a depth first search
sys.setrecursionlimit(30000)

input_file = 'Misc data - no times/med2'
img = cv2.imread(input_file+'.jpg')
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
width, height = img.shape
img_new = cv2.resize(img, (height//2,width//2))
width1, height1 = img_new.shape
# flatten image
img_ = img_new.reshape(-1,1)

kmeans = KMeans(n_clusters=2).fit(img_)
values = kmeans.cluster_centers_.squeeze()
labels = kmeans.labels_
img_compressed = np.choose(labels, values)
labels_reshaped = labels.reshape(width1, height1)
labels_reshaped = labels_reshaped.astype(np.uint8)

plt.imshow(labels_reshaped)
plt.show()

label1 = labels_reshaped[0, 0]  # Get the label of the top-left corner pixel
binary_image = np.where(labels_reshaped == label1, 0, 255).astype(np.uint8)

plt.imshow(binary_image, cmap='gray')
plt.axis('off')  
plt.show()

In [14]:
output_file = 'binary_'
cv2.imwrite(output_file + input_file + '.png', binary_image)
print(f"Binary image saved to {output_file}")

Binary image saved to binary_


In [ ]:
import cv2
test_image = np.asarray(labels_reshaped, dtype=np.uint8)
plt.imshow(test_image)
threshold_value = 225
_, binary_image = cv2.threshold(test_image, threshold_value, 255, cv2.THRESH_BINARY)
binary_image_path = 'binary_image_med1.png'
cv2.imwrite(binary_image_path, binary_image)

In [ ]:
def perform_opening(labels_reshaped, kernel_size):
    labels_reshaped = labels_reshaped.astype(np.uint8)

    kernel = np.ones((kernel_size, kernel_size), np.uint8)

    opened_image = cv2.morphologyEx(labels_reshaped, cv2.MORPH_OPEN, kernel)

    return opened_image

kernel_size = 5
opened_image = perform_opening(labels_reshaped, kernel_size)

contours, _ = cv2.findContours(opened_image.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
longest_contour = max(contours, key=cv2.contourArea)

foreground_mask = np.zeros_like(img, dtype=np.uint8)
cv2.drawContours(foreground_mask, [longest_contour], -1, 255, cv2.FILLED)

y_values = np.where(foreground_mask != 0)[0]
highest_y = min(y_values)

x, y, w, h = cv2.boundingRect(longest_contour)
background_cropped = img_new[0:highest_y]

plt.imshow(background_cropped)

In [ ]:
selected_point = None

def shift_image_center(image, center):
    rows, cols = image.shape[:2]
    dx, dy = cols/2 - center[0], rows/2 - center[1]
    translation_matrix = np.float32([[1, 0, dx], [0, 1, dy]])
    return cv2.warpAffine(image, translation_matrix, (cols, rows))

def mouse_callback(event, x, y, flags, param):
    global selected_point
    if event == cv2.EVENT_LBUTTONDOWN:
        selected_point = (x, y)
        cv2.destroyAllWindows()

cv2.imshow("Select New Center", background_cropped)
cv2.setMouseCallback("Select New Center", mouse_callback)
cv2.waitKey(0)

if selected_point is not None:
    shifted_image = shift_image_center(background_cropped, selected_point)

    plt.imshow(shifted_image)
    plt.show()
else:
    print("No point selected.")